In [3]:
#create 1d cnn model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN1d(nn.Module):
    #create a 1d cnn regression model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, kernel_size, layer_num, hidden_size,
                  dropout=0.5, batch_norm=True):
        #input shape: (batch_size, input_len, input_dim)
        super(CNN1d, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.kernel_size = kernel_size
        self.layer_num = layer_num
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.batch_norm = batch_norm
        self.convs = nn.ModuleList([nn.Conv1d(self.input_dim, self.hidden_size, self.kernel_size, padding=self.kernel_size // 2)])
        self.bns = nn.ModuleList([nn.BatchNorm1d(self.hidden_size)])
        for i in range(self.layer_num - 1):
            self.convs.append(nn.Conv1d(self.hidden_size, self.hidden_size, self.kernel_size, padding=self.kernel_size // 2))
            self.bns.append(nn.BatchNorm1d(self.hidden_size))

        self.fc1 = nn.Linear(self.input_len * self.hidden_size, 74)
        self.dropout = nn.Dropout(self.dropout)
        self.init_weights()

    def init_weights(self):
        #initialize weights
        for conv in self.convs:
            torch.nn.init.xavier_uniform_(conv.weight)
        torch.nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        #forward pass
        #input shape: (batch_size, input_len, input_dim)
        #output shape: (batch_size, 1)
        x = x.transpose(1, 2)
        for i, conv in enumerate(self.convs):
            if self.batch_norm:
                x = F.relu(self.bns[i](conv(x)))
            else:
                x = F.relu(conv(x))
            x = self.dropout(x)

        x = x.view(-1, (self.input_len * self.hidden_size))
        x = self.fc1(x)

        return x
    
    def predict(self, x):
        #predict the target value
        return self.forward(x)
    
    def loss(self, x, y):
        #calculate loss
        return F.mse_loss(self.forward(x), y)




In [4]:
#positional encoding
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class PositionalEncoding(nn.Module):
    #add positional encoding to the input
    def __init__(self, input_len, input_dim, dropout=0.5):
        super(PositionalEncoding, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.dropout = nn.Dropout(dropout)
        self.pe = torch.zeros(self.input_len, self.input_dim)
        position = torch.arange(0, self.input_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.input_dim, 2).float() * (-math.log(10000.0) / self.input_dim))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)
        self.pe.requires_grad = False

    def forward(self, x):
        #forward pass
        #input shape: (batch_size, input_len, input_dim)
        #output shape: (batch_size, input_len, input_dim)
        x = x + self.pe
        return self.dropout(x)


In [5]:
class Attention(nn.Module):
    #create MULTIHEADATTENTION model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, hidden_size, num_heads, layer_num, dropout=0.5):
        super(Attention, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.layer_num = layer_num
        self.dropout = dropout
        #embedding

        self.embedding = nn.Linear(self.input_dim, self.hidden_size)
        self.q_linears = nn.ModuleList([nn.Linear(self.hidden_size, self.hidden_size) for i in range(self.layer_num)])
        self.k_linears = nn.ModuleList([nn.Linear(self.hidden_size, self.hidden_size) for i in range(self.layer_num)])
        self.v_linears = nn.ModuleList([nn.Linear(self.hidden_size, self.hidden_size) for i in range(self.layer_num)])
        self.multihead_attns = nn.ModuleList([nn.MultiheadAttention(self.hidden_size, self.num_heads) for i in range(self.layer_num)])
        self.norms = nn.ModuleList([nn.BatchNorm1d(self.input_len) for i in range(self.layer_num)])
        self.out = nn.Linear(self.hidden_size * self.input_len, 74)

        self.dropout = nn.Dropout(dropout)
        self.pe = torch.zeros(self.input_len, self.hidden_size)
        position = torch.arange(0, self.input_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.hidden_size, 2).float() * (-math.log(10000.0) / self.hidden_size))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)
        self.pe.requires_grad = False
        self.init_weights()
        self.mean = nn.Parameter(torch.zeros(74))
        self.std = nn.Parameter(torch.ones(74))

    def init_weights(self):
        #initialize weights
        torch.nn.init.xavier_uniform_(self.embedding.weight)
        torch.nn.init.xavier_uniform_(self.out.weight)
        
    def positional_encoding(self, x):
        #use positional encoding
        x = x + self.pe.to(x.device)
        return self.dropout(x)

    def forward(self, x):
        x = self.embedding(x)
        #use positional encoding
        x = self.positional_encoding(x)
        #forward pass
        bs = x.size(0)
        for i in range(self.layer_num):
            q = self.q_linears[i](x)
            k = self.k_linears[i](x)
            v = self.v_linears[i](x)
            x, _ = self.multihead_attns[i](q, k, v)
            x = self.norms[i](x)
            x = self.dropout(x)
        x = x.view(bs, -1)
        x = self.out(x)
        x = (x - self.mean) / self.std
        return x

    def loss(self, x, y):
        #calculate loss
        return F.mse_loss(self.forward(x), y)
    


In [6]:

num_samples = 1000
#test the model
X = torch.randn(num_samples, 4 * 74)
Y = torch.randn(num_samples, 74)


In [7]:
#data loader
day = torch.randint(0, 30, (num_samples, 1))
class MyDataset(torch.utils.data.Dataset):

    def __init__(self, X, y, day, input_len=128):
        #the input data is a 1d array, indicate the minute of the day
        self.X = X
        self.y = y
        self.day = day
        self.input_len = input_len

    def __getitem__(self, index):
        #output previous self.input_len minutes data and target value
        #if the there is no enough data in the same day, pad with 0
        d = self.day[index]
        start = index - self.input_len
        if start < 0:
            start = 0
        if self.day[start] != d:
            while self.day[start] != d:
                start += 1
            #pad with 0 before start
        if index - start < self.input_len:
            x = torch.zeros(self.input_len, self.X.shape[1])
            x[self.input_len - index + start: self.input_len] = self.X[start: index].clone()
        else:
            x = self.X[start: index]
            
        y = self.y[index].clone()
        if x.shape != (self.input_len, self.X.shape[1]):
            print(x.shape, index, start)
        return x, y
    def __len__(self):
        return len(self.X)
    
mydataset = MyDataset(X, Y, day, input_len=120)
train_index = range(0, int(num_samples * 0.8))
test_index = range(0, int(num_samples * 0.8))
#test_index = range(int(num_samples * 0.8), num_samples)

train_dataset = torch.utils.data.Subset(mydataset, train_index)
test_dataset = torch.utils.data.Subset(mydataset, test_index)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)



In [8]:
#train the model with 4 GPUs
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import DataParallel

model = CNN1d(input_len=120, input_dim=74, kernel_size=3, layer_num=3, hidden_size=128, dropout=0.5, batch_norm=True)
def correlation_t(x, y):
    return torch.sum(x * y) / (torch.sqrt(torch.sum(x * x)) * torch.sqrt(torch.sum(y * y)))

optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
for epoch in range(100):
    prediction = []
    ground_truth = []
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        p = model(x)
        loss = F.mse_loss(p, y) + 0.01 * torch.sum(model.fc1.weight ** 2)
        prediction.append(p)
        ground_truth.append(y)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(epoch, i, loss.item())
    prediction = torch.cat(prediction, dim=0)
    ground_truth = torch.cat(ground_truth, dim=0)
    print(prediction.shape, ground_truth.shape)
    print(epoch, correlation_t(prediction, ground_truth).item())

#the number of gpu 
print(torch.cuda.device_count())



RuntimeError: Given groups=1, weight of size [128, 74, 3], expected input[128, 296, 120] to have 74 channels, but got 296 channels instead

In [8]:
class MLP(nn.Module):
    #create MLP model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, hidden_size, layer_num, dropout=0.5):
        super(MLP, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.layer_num = layer_num
        self.dropout = dropout
        #embedding
        self.linears = nn.ModuleList([nn.Linear(self.input_dim, self.hidden_size)])
        self.norms = nn.ModuleList([nn.BatchNorm1d(self.input_len)])

        for i in range(self.layer_num - 1):
            self.linears.append(nn.Linear(self.hidden_size, self.hidden_size))
            self.norms.append(nn.BatchNorm1d(self.input_len))

        self.out = nn.Linear(self.hidden_size * self.input_len, 74)
        self.dropout = nn.Dropout(dropout)
        self.init_weights()
        self.mean = nn.Parameter(torch.zeros(74))
        self.std = nn.Parameter(torch.ones(74))

    def init_weights(self):
        #initialize weights
        torch.nn.init.xavier_uniform_(self.embedding.weight)
        torch.nn.init.xavier_uniform_(self.out.weight)

    def forward(self, x):
        #forward pass
        bs = x.size(0)

        for i in range(self.layer_num):
            x = self.linears[i](x)
            x = self.norms[i](x)
            x = self.dropout(x)
            x = F.relu(x)

        x = x.view(bs, -1)
        x = self.out(x)
        x = (x - self.mean) / self.std
        return x


In [16]:
class RNN(nn.Module):
    #create RNN model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, hidden_size, layer_num, dropout=0.5, bidirectional=True):
        super(RNN, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.layer_num = layer_num
        self.dropout = dropout
        #embedding
        self.bidirectional = bidirectional
        self.rnn = nn.GRU(self.input_dim, self.hidden_size, self.layer_num, batch_first=True, dropout=self.dropout, bidirectional = self.bidirectional)
        self.out = nn.Linear(self.hidden_size * (1 + int(self.bidirectional)), 74)
        self.dropout = nn.Dropout(dropout)
        self.init_weights()


    def init_weights(self):
        #initialize weights
        torch.nn.init.xavier_uniform_(self.out.weight)

    def forward(self, x):
        #forward pass
        bs = x.size(0)
        x, hn = self.rnn(x)
        h = hn[-(1 + int(self.bidirectional)):]
        x = torch.cat(h.split(1), dim=-1).squeeze(0)
        x = self.out(x)

        return x

In [17]:
model = RNN(input_len=120, input_dim=296, hidden_size=128, layer_num=3, dropout=0.5)
def correlation_t(x, y):
    return torch.sum(x * y) / (torch.sqrt(torch.sum(x * x)) * torch.sqrt(torch.sum(y * y)))

optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
for epoch in range(100):
    prediction = []
    ground_truth = []
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        p = model(x)
        loss = F.mse_loss(p, y)
        prediction.append(p)
        ground_truth.append(y)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(epoch, i, loss.item())
    prediction = torch.cat(prediction, dim=0)
    ground_truth = torch.cat(ground_truth, dim=0)
    print(prediction.shape, ground_truth.shape)
    print(epoch, correlation_t(prediction, ground_truth).item())

0 0 1.0544946193695068


KeyboardInterrupt: 

In [22]:
class LSTM(nn.Module):
    #create LSTM model
    #the ouput is a target value of 60 minutes later, is a scalar
    def __init__(self, input_len, input_dim, hidden_size, layer_num, dropout=0.5, bidirectional=True):
        super(LSTM, self).__init__()
        self.input_len = input_len
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.layer_num = layer_num
        self.dropout = dropout
        #embedding
        self.bidirectional = bidirectional
        self.lstm = nn.LSTM(self.input_dim, self.hidden_size, self.layer_num, batch_first=True, dropout=self.dropout, bidirectional = self.bidirectional)
        self.out = nn.Linear(self.hidden_size * (1 + int(self.bidirectional)), 74)
        self.dropout = nn.Dropout(dropout)
        self.init_weights()


    def init_weights(self):
        #initialize weights
        torch.nn.init.xavier_uniform_(self.out.weight)

    def forward(self, x):
        #forward pass
        bs = x.size(0)
        _, (hn, _)= self.lstm(x)
        h = hn[-(1 + int(self.bidirectional)):]
        x = torch.cat(h.split(1), dim=-1).squeeze(0)
        x = self.out(x)

        return x

In [23]:
model = LSTM(input_len=120, input_dim=296, hidden_size=128, layer_num=3, dropout=0.5)
def correlation_t(x, y):
    return torch.sum(x * y) / (torch.sqrt(torch.sum(x * x)) * torch.sqrt(torch.sum(y * y)))

optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
for epoch in range(100):
    prediction = []
    ground_truth = []
    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        p = model(x)
        loss = F.mse_loss(p, y)
        prediction.append(p)
        ground_truth.append(y)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(epoch, i, loss.item())
    prediction = torch.cat(prediction, dim=0)
    ground_truth = torch.cat(ground_truth, dim=0)
    print(prediction.shape, ground_truth.shape)
    print(epoch, correlation_t(prediction, ground_truth).item())

0 0 1.031086802482605


KeyboardInterrupt: 

In [87]:
#tree models random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

X = torch.randn(num_samples, 4 * 74)
Y = torch.randn(num_samples, 1)
X = X.numpy()
Y = Y.numpy()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
#multiprocessing
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)


/var/folders/gw/rdd16gz922j8hc9s9c737lpw0000gr/T/ipykernel_67180/38055778.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=0)

In [26]:
#linear classifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

X = torch.randn(num_samples, 4 * 74)
# 4 classes
Y = torch.randint(0, 4, (num_samples, 1))
X = X.numpy()
Y = Y.numpy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
clf = SGDClassifier(loss="log", penalty="l2", max_iter=1000)


/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.23

In [27]:
#'04:00:00' to int
def time_to_int(time):
    #convert time to int
    #input time format: '04:00:00'
    #output int
    h, m, s = time.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

#int to '04:00:00'
def int_to_time(i):
    #convert int to time
    #input int
    #output time format: '04:00:00'
    h = i // 3600
    m = (i - h * 3600) // 60
    s = i - h * 3600 - m * 60
    return '%02d:%02d:%02d' % (h, m, s)

#convert time to int
a = time_to_int('04:00:00')
a -= 1
print(int_to_time(a))

03:59:59


In [78]:
#the std of the previous 60 minutes
#nan to 0 in np array
#inf to 0 in np array
#-inf to 0 in np array
x = np.array([1, 2, 3, np.nan])
x[np.isnan(x)] = 0
x[np.isinf(x)] = 0
x[np.isneginf(x)] = 0


In [79]:

df = pd.DataFrame([1, 1, np.inf, np.nan, -np.inf])
df.replace([np.inf, -np.inf], 0, inplace=True)

In [62]:
#for each row, normalize the mean and std of the previous 100 rows
df = pd.DataFrame(np.random.randn(1000, 3))
std = df.rolling(100).std().fillna(df.std())
mean = df.rolling(100).mean().fillna(df.mean())
df = (df - mean) / std



In [85]:
np.concatenate([np.zeros((1, 2)), np.ones((1, 2))])

array([[0., 0.],
       [1., 1.]])

In [70]:

df = pd.DataFrame(np.random.randn(1000, 3))
#assign [1, 5, 6] to each row
df.values[:] = [1, 5, 6]

In [88]:
#XGBoost Regressor

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor
import numpy as np

X = torch.randn(num_samples, 4 * 74)
Y = torch.randn(num_samples, 1)
X = X.numpy()
Y = Y.numpy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
clf = HistGradientBoostingRegressor(max_iter=1000, learning_rate=0.1, max_depth=10, random_state=0)
clf.fit(X_train, y_train)



/opt/homebrew/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingRegressor(max_depth=10, max_iter=1000, random_state=0)

In [90]:
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = torch.randn(num_samples, 4 * 74)
Y = torch.randn(num_samples, 1)
X = X.numpy()
Y = Y.numpy()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
clf = XGBRegressor(max_depth=10, learning_rate=0.1, n_estimators=100, random_state=0)
clf.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=0, ...)

In [89]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.6 MB/s eta 0:00:00a 0:00:01
